## 📋 Setup & Installation

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install wandb

In [3]:
import os
import json
import torch
import wandb
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

# Check GPU
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-02 03:44:07.935717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764647048.184811      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764647048.243987      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.83 GB


## 🔐 WandB Login (for monitoring)

In [5]:
# Login to WandB for experiment tracking
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Login with API key from Kaggle Secrets
wandb.login(key=wandb_api_key)


# Initialize WandB project with GRPO synthetic data config
wandb.init(
    project="vietnamese-legal-ai-grpo",
    name="llama3.2-3b-grpo-synthetic-sft-v1",
    config={
        "base_model": "mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo",
        "dataset": "synthetic_legal_qa_grpo_training.jsonl",
        "task": "structured_legal_qa",
        "language": "vietnamese",
        "format": "grpo_structured",
        "max_seq_length": 2048,  # Increased for structured format
        "lora_r": 16,  # Reduced since base model already fine-tuned
        "lora_alpha": 16,
        "learning_rate": 1e-4,  # Lower LR for already fine-tuned model
        "num_epochs": 1,  # Less epochs needed
        "batch_size": 2,
        "gradient_accumulation": 8,
        "effective_batch_size": 16,
    },
    settings=wandb.Settings(
        _disable_meta=False,
        _disable_stats=False,
    )
)

print("✅ WandB initialized for GRPO synthetic data training")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mikeethanh04 (mikeethanh04-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ WandB initialized for GRPO synthetic data training


## ⚙️ Model Configuration - GRPO Merged Model

### Tại sao sử dụng GRPO merged model?
- ✅ **Already GRPO trained**: Model đã được train với GRPO format
- ✅ **Structured reasoning**: Đã biết format `<start_working_out>` và `<SOLUTION>`
- ✅ **Domain adapted**: Đã fine-tune trên legal domain
- ✅ **Consistent format**: Sẽ dễ dàng học synthetic data cùng format
- ✅ **Less training needed**: Chỉ cần ít epochs để adapt với synthetic data

In [6]:
# Model configuration for GRPO merged model
max_seq_length = 2048  # Increased for structured format with reasoning
dtype = None  # Auto-detect. Use Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage

# Load the GRPO merged model
model_name = "mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1"  # Your GRPO merged model

print(f"🔄 Loading GRPO merged model: {model_name}")
print("⚠️ This model already contains GRPO training adaptations")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print(f"✅ GRPO merged model loaded: {model_name}")
print(f"📏 Max sequence length: {max_seq_length}")
print(f"🔢 4-bit quantization: {load_in_4bit}")

🔄 Loading GRPO merged model: mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1
⚠️ This model already contains GRPO training adaptations
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/943 [00:00<?, ?B/s]

✅ GRPO merged model loaded: mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v1
📏 Max sequence length: 2048
🔢 4-bit quantization: True


## 🎯 LoRA Configuration - Lighter for Already Fine-tuned Model

### LoRA Parameters cho model đã fine-tune:
- **r (rank)**: 8-16 thay vì 32 (model đã có knowledge)
- **lora_alpha**: Tương ứng với r
- **learning_rate**: Thấp hơn (5e-5 đến 1e-4)
- **epochs**: 1-2 epochs thay vì 3+

In [7]:
# Apply LoRA adapters với settings nhẹ hơn cho model đã fine-tune
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # Reduced from 32 since model is already fine-tuned
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # All attention & MLP layers
    lora_alpha=32,  # Equal to r
    lora_dropout=0,  # 0 is optimized by Unsloth
    bias="none",  # "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's long context support
    random_state=3407,  # For reproducibility
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None,  # LoftQ quantization
)

print("✅ LoRA adapters applied (lighter config for pre-trained model)")
print(f"📊 Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"💡 Trainable ratio: {100 * sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters()):.2f}%")

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


✅ LoRA adapters applied (lighter config for pre-trained model)
📊 Trainable parameters: 48,627,712
📊 Total parameters: 1,852,091,392
💡 Trainable ratio: 2.63%


## 📊 GRPO Synthetic Data Preparation

### Expected Data Format:
```json
{
  "messages": [
    {"role": "system", "content": "System prompt với GRPO format"},
    {"role": "user", "content": "Câu hỏi pháp luật"},
    {"role": "assistant", "content": "<start_working_out>\nSuy nghĩ...\n<end_working_out>\n\n<SOLUTION>Câu trả lời</SOLUTION>"}
  ]
}
```

In [11]:
# Load GRPO synthetic data
# Update path to your synthetic data file
data_path = "/kaggle/input/synthetic-legal-2k/synthetic_legal_qa_grpo_format.jsonl"  # Correct path

print(f"🔍 Looking for GRPO synthetic data at: {data_path}")

# Check if file exists
if not os.path.exists(data_path):
    print(f"⚠️ Data file not found at {data_path}")
    print("📥 Please check the path or upload the file")
    
    # Alternative: Try different common paths
    alternative_paths = [
        "/content/synthetic_legal_qa_grpo_format.jsonl",  # Colab
        "/kaggle/input/grpo-synthetic-data/synthetic_legal_qa_grpo_format.jsonl",  # Kaggle
        "synthetic_legal_qa_grpo_format.jsonl",  # Current directory
    ]
    
    for alt_path in alternative_paths:
        if os.path.exists(alt_path):
            data_path = alt_path
            print(f"✅ Found data at alternative path: {data_path}")
            break
    else:
        print("❌ Please ensure the GRPO synthetic data file is available")
        raise FileNotFoundError(f"Data file not found: {data_path}")

# Load JSONL data and convert to messages format
data = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        # Convert from question/answer format to messages format
        messages_item = {
            "messages": [
                {
                    "role": "system", 
                    "content": "Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy: 1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out> 2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION>"
                },
                {
                    "role": "user", 
                    "content": item["question"]
                },
                {
                    "role": "assistant", 
                    "content": item["answer"]
                }
            ]
        }
        data.append(messages_item)

print(f"\n📊 Loaded {len(data):,} GRPO training samples")
print(f"📊 Converted from question/answer format to messages format")

# Show sample
print("\n📝 Sample GRPO data structure:")
sample = data[0]
print(f"Keys: {list(sample.keys())}")
if "messages" in sample:
    print(f"\nMessages structure:")
    for i, msg in enumerate(sample["messages"]):
        content_preview = msg["content"][:100] + "..." if len(msg["content"]) > 100 else msg["content"]
        print(f"  {i+1}. {msg['role']}: {content_preview}")
        
# Validate GRPO format
grpo_format_count = 0
for item in data[:100]:  # Check first 100 samples
    if "messages" in item:
        for msg in item["messages"]:
            if msg["role"] == "assistant":
                if "<start_working_out>" in msg["content"] and "<SOLUTION>" in msg["content"]:
                    grpo_format_count += 1
                break

print(f"\n✅ GRPO format validation: {grpo_format_count}/100 samples have proper structure")
if grpo_format_count >= 50:
    print("🎉 Excellent! Data has good GRPO format compliance")
else:
    print("⚠️ Warning: Low GRPO format compliance. Check data generation.")

🔍 Looking for GRPO synthetic data at: /kaggle/input/synthetic-legal-2k/synthetic_legal_qa_grpo_format.jsonl

📊 Loaded 2,121 GRPO training samples
📊 Converted from question/answer format to messages format

📝 Sample GRPO data structure:
Keys: ['messages']

Messages structure:
  1. system: Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy: 1. Suy nghĩ và ph...
  2. user: Cổng thông tin điện tử Bộ Tư pháp là gì?
  3. assistant: <start_working_out>
Câu hỏi của người dùng mong muốn tìm hiểu về Cổng thông tin điện tử Bộ Tư pháp, ...

✅ GRPO format validation: 100/100 samples have proper structure
🎉 Excellent! Data has good GRPO format compliance


In [12]:
# Split data: 85% train, 10% validation, 5% test (less aggressive split for synthetic data)
train_data, temp_data = train_test_split(data, test_size=0.15, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.33, random_state=42)  # 5% test, 10% val

print(f"📊 Train: {len(train_data):,} samples ({len(train_data)/len(data)*100:.1f}%)")
print(f"📊 Validation: {len(val_data):,} samples ({len(val_data)/len(data)*100:.1f}%)")
print(f"📊 Test: {len(test_data):,} samples ({len(test_data)/len(data)*100:.1f}%)")

📊 Train: 1,802 samples (85.0%)
📊 Validation: 213 samples (10.0%)
📊 Test: 106 samples (5.0%)


## 📝 Chat Template cho GRPO Format

Data đã có format messages, chỉ cần convert sang text format cho training:

In [15]:
# Convert to HuggingFace Dataset and format for Unsloth
print("🔄 Converting data to HuggingFace Dataset format...")

def format_chat_for_unsloth(examples):
    """Format chat data for Unsloth training"""
    texts = []
    
    # Handle batch processing
    if isinstance(examples, dict) and "messages" in examples:
        # Single example
        examples = [examples]
    elif isinstance(examples, dict) and isinstance(examples.get("messages", []), list) and len(examples["messages"]) > 0 and isinstance(examples["messages"][0], list):
        # Batch of examples where examples["messages"] is a list of message lists
        examples = [{"messages": msg_list} for msg_list in examples["messages"]]
    elif isinstance(examples, list):
        # Already a list of examples
        pass
    else:
        # Convert from dataset format
        if "messages" in examples and isinstance(examples["messages"], list):
            if len(examples["messages"]) > 0 and isinstance(examples["messages"][0], list):
                # Batch format: examples["messages"] = [msg_list1, msg_list2, ...]
                examples = [{"messages": msg_list} for msg_list in examples["messages"]]
            else:
                # Single example wrapped in batch
                examples = [examples]
    
    for example in examples:
        messages = example["messages"]
        
        # Extract parts from messages
        system_msg = ""
        user_msg = ""
        assistant_msg = ""
        
        for msg in messages:
            if msg["role"] == "system":
                system_msg = msg["content"]
            elif msg["role"] == "user":
                user_msg = msg["content"]
            elif msg["role"] == "assistant":
                assistant_msg = msg["content"]
        
        # Format for Unsloth training (ChatML-like format)
        conversation = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{assistant_msg}<|eot_id|>"""
        
        texts.append(conversation)
    
    return {"text": texts}

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

print("✅ Datasets created successfully")
print(f"📊 Train dataset: {len(train_dataset)} samples")
print(f"📊 Validation dataset: {len(val_dataset)} samples") 
print(f"📊 Test dataset: {len(test_dataset)} samples")

# Format for Unsloth training
print("🔄 Formatting data for Unsloth training...")

train_dataset = train_dataset.map(
    format_chat_for_unsloth, 
    batched=True,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    format_chat_for_unsloth, 
    batched=True,
    remove_columns=val_dataset.column_names
)

print("✅ Data formatted for Unsloth training")
print("\n📝 Example formatted conversation:")
print(train_dataset[0]['text'][:800] + "...")

# Show token statistics
print("\n📊 Dataset Statistics:")
print(f"📏 Train samples: {len(train_dataset)}")
print(f"📏 Val samples: {len(val_dataset)}")
if len(train_dataset) > 0:
    sample_text = train_dataset[0]['text']
    sample_tokens = len(tokenizer.encode(sample_text))
    print(f"📏 Sample text length: {sample_tokens} tokens")
    print(f"📏 Estimated max tokens needed: {sample_tokens}")

    # Check if sequence is too long
    if sample_tokens > max_seq_length:
        print(f"⚠️ Warning: Sample length ({sample_tokens}) exceeds max_seq_length ({max_seq_length})")
        print("💡 Consider increasing max_seq_length or truncating data")

🔄 Converting data to HuggingFace Dataset format...
✅ Datasets created successfully
📊 Train dataset: 1802 samples
📊 Validation dataset: 213 samples
📊 Test dataset: 106 samples
🔄 Formatting data for Unsloth training...


Map:   0%|          | 0/1802 [00:00<?, ? examples/s]

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

✅ Data formatted for Unsloth training

📝 Example formatted conversation:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy: 1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out> 2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION><|eot_id|><|start_header_id|>user<|end_header_id|>

Quy định về tốc độ tối đa cho xe cơ giới trên đường cao tốc là gì?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<start_working_out>
Người dùng muốn biết quy định về tốc độ tối đa cho xe cơ giới trên đường cao tốc. Để trả lời câu hỏi này, tôi cần tìm hiểu các quy định liên quan đến tốc độ tối đa trên đường cao tốc trong luật giao thông Việt Nam. Văn bản pháp luật không chứa thông tin cụ thể về tốc độ tối đa, tuy nhiên, theo Luật Giao thông đườn...

📊 Dataset Statistics:
📏 Train samples: 1802
📏 Val samples: 213
📏 Sample text length: 331 tokens
📏 Estimated max tokens nee

## 🎓 Training Configuration - Optimized for GRPO Synthetic Data

### Settings cho model đã fine-tune + synthetic data:
- **Epochs**: 1-2 (model đã có base knowledge)
- **Learning rate**: 5e-5 đến 1e-4 (thấp hơn)
- **Batch size**: Nhỏ hơn do sequence dài hơn
- **More evaluation**: Monitor overfitting carefully

In [16]:
# Training arguments optimized for GRPO synthetic data
training_args = TrainingArguments(
    # Output & Logging
    output_dir="./outputs-grpo-synthetic",
    run_name="llama3.2-3b-grpo-synthetic-v1",
    
    # Training dynamics - Conservative for already fine-tuned model
    num_train_epochs=2,  # Start with 1 epoch, can increase if needed
    per_device_train_batch_size=4,  # Reduced due to longer sequences
    gradient_accumulation_steps=4,  # Increased to maintain effective batch size = 16
    
    # Optimization - Lower LR for already fine-tuned model
    optim="adamw_8bit",  # 8-bit AdamW for memory efficiency
    learning_rate=2e-4,  # Lower LR than fresh model (was 2e-4)
    weight_decay=0.01,
    warmup_ratio=0.05,  # Shorter warmup (5% instead of 10%)
    lr_scheduler_type="cosine",  # Cosine annealing
    
    # Performance
    fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 for T4
    bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if supported
    
    # Logging & Saving - More frequent for careful monitoring
    logging_steps=5,  # Very frequent logging for synthetic data
    logging_strategy="steps",
    logging_first_step=True,
    save_strategy="steps",
    save_steps=25,  # Save very frequently to avoid overfitting
    save_total_limit=3,
    
    # Evaluation - Very frequent to catch overfitting early
    eval_strategy="steps",
    eval_steps=25,  # Frequent evaluation
    eval_accumulation_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Early stopping to prevent overfitting on synthetic data
    # early_stopping_patience=3,
    
    # WandB integration
    report_to="wandb",
    logging_nan_inf_filter=True,
    include_inputs_for_metrics=False,
    
    # Progress bar
    disable_tqdm=False,
    log_level="info",
    dataloader_num_workers=2,
)

print("✅ Training arguments configured for GRPO synthetic data")
print(f"💾 Per device batch size: {training_args.per_device_train_batch_size}")
print(f"📊 Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"📚 Learning rate: {training_args.learning_rate} (reduced for fine-tuned model)")
print(f"📊 Epochs: {training_args.num_train_epochs} (conservative approach)")
print(f"🕐 Total training steps: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

✅ Training arguments configured for GRPO synthetic data
💾 Per device batch size: 4
📊 Effective batch size: 16
📚 Learning rate: 0.0002 (reduced for fine-tuned model)
📊 Epochs: 2 (conservative approach)
🕐 Total training steps: 224


In [17]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Keep False for structured GRPO format
    args=training_args,
)

print("✅ SFT Trainer initialized for GRPO synthetic data")

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1802 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/213 [00:00<?, ? examples/s]

Using auto half precision backend


✅ SFT Trainer initialized for GRPO synthetic data


## 🚀 Start Training!

**Estimated time**: ~30-60 min for 1 epoch (shorter due to pre-trained model)  
**Memory usage**: ~15-16GB VRAM (longer sequences)  
**Watch for**: Overfitting (eval loss increasing while train loss decreasing)

In [18]:
# Show GPU stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"🖥️ GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"💾 {start_gpu_memory} GB of memory reserved.")
print(f"🎯 Training GRPO model with synthetic structured data...")

# Start training
print("\n🚀 Starting GRPO synthetic data training...\n")
trainer_stats = trainer.train()

# Show final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)

print("\n" + "="*50)
print("✅ GRPO SYNTHETIC DATA TRAINING COMPLETED!")
print("="*50)
print(f"⏱️ Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"💾 Peak reserved memory: {used_memory} GB")
print(f"📊 Memory used for training: {used_memory_for_lora} GB")
print(f"📈 Percentage of max memory: {used_percentage}%")
print(f"🎯 Final train loss: {trainer_stats.metrics['train_loss']:.4f}")

🖥️ GPU = Tesla T4. Max memory = 14.741 GB.
💾 3.072 GB of memory reserved.
🎯 Training GRPO model with synthetic structured data...

🚀 Starting GRPO synthetic data training...



The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: attention_mask, text. If attention_mask, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
skipped Embedding(128256, 3072, padding_idx=128004): 375.75M params
skipped: 375.75M params
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,802 | Num Epochs = 2 | Total steps = 114
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
25,0.710300,0.703001
50,0.670900,0.642131
75,0.589700,0.616110
100,0.584800,0.604359


The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: attention_mask, text. If attention_mask, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 213
  Batch size = 8
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Saving model checkpoint to ./outputs-grpo-synthetic/checkpoint-25
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mikeethanh--vietnamese-legal-llama3.2-3b-merged-sft-v1/snapshots/2e53eb44a6fae3e8d7c3d7760f2b9163383c18c5/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,

eval/loss,█▄▂▁
eval/runtime,█▁▃▂
eval/samples_per_second,▁█▆▇
eval/steps_per_second,▁█▇▇
train/epoch,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇██
train/global_step,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇██
train/grad_norm,▆█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▆████▇▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
train/loss,█▇▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.60436
eval/runtime,56.1082



✅ GRPO SYNTHETIC DATA TRAINING COMPLETED!
⏱️ Training time: 2371.59 seconds
💾 Peak reserved memory: 6.746 GB
📊 Memory used for training: 3.674 GB
📈 Percentage of max memory: 45.764%
🎯 Final train loss: 0.6966


## 📊 Evaluation

In [19]:
# Evaluate on validation set
print("📊 Evaluating on validation set...\n")
eval_results = trainer.evaluate()

print("="*50)
print("VALIDATION RESULTS - GRPO SYNTHETIC")
print("="*50)
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

# Log to WandB
wandb.log({
    "final_eval_loss": eval_results['eval_loss'],
    "model_type": "grpo_synthetic_sft"
})

The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: attention_mask, text. If attention_mask, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 213
  Batch size = 8


📊 Evaluating on validation set...



Error: You must call wandb.init() before wandb.log()

## 🧪 Inference Testing - GRPO Format Validation

In [20]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# GRPO format markers for validation
reasoning_start = "<start_working_out>"
reasoning_end = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

def test_grpo_model(user_message, max_new_tokens=512):
    """Test model with GRPO system prompt"""
    
    # GRPO system prompt
    system_prompt = f"""Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Câu trả lời cần dựa trên quy định pháp luật hiện hành và phải rõ ràng, dễ hiểu."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    # Apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # Extract only the response part
    if "<|start_header_id|>assistant<|end_header_id|>" in response:
        response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        response = response.split("<|eot_id|>")[0].strip()
    
    return response

def validate_grpo_format(response):
    """Validate if response follows GRPO format"""
    has_reasoning = reasoning_start in response and reasoning_end in response
    has_solution = solution_start in response and solution_end in response
    
    return {
        "has_reasoning": has_reasoning,
        "has_solution": has_solution,
        "proper_format": has_reasoning and has_solution
    }

# Test questions
test_questions = [
    "Mức phạt cho việc vượt đèn đỏ đối với xe máy là bao nhiêu?",
    "Điều kiện để được cấp bằng lái xe ô tô hạng B1 là gì?",
    "Hành vi nào bị cấm khi tham gia giao thông đường bộ?"
]

print("🧪 Testing GRPO model with structured format validation...\n")
print("="*80)

format_validation_results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"TEST QUESTION #{i}")
    print(f"{'='*80}")
    print(f"\n❓ Question: {question}")
    
    # Generate response
    response = test_grpo_model(question)
    print(f"\n🤖 Model Response:\n{response}")
    
    # Validate format
    validation = validate_grpo_format(response)
    format_validation_results.append(validation)
    
    print(f"\n📊 Format Validation:")
    print(f"   Has reasoning section: {validation['has_reasoning']} ✅" if validation['has_reasoning'] else f"   Has reasoning section: {validation['has_reasoning']} ❌")
    print(f"   Has solution section: {validation['has_solution']} ✅" if validation['has_solution'] else f"   Has solution section: {validation['has_solution']} ❌")
    print(f"   Proper GRPO format: {validation['proper_format']} ✅" if validation['proper_format'] else f"   Proper GRPO format: {validation['proper_format']} ❌")
    print(f"\n{'='*80}")

# Summary of format validation
proper_format_count = sum(1 for r in format_validation_results if r['proper_format'])
print(f"\n📊 GRPO FORMAT VALIDATION SUMMARY:")
print(f"   Proper format: {proper_format_count}/{len(test_questions)} ({proper_format_count/len(test_questions)*100:.1f}%)")

if proper_format_count == len(test_questions):
    print("🎉 Excellent! Model consistently follows GRPO format")
elif proper_format_count >= len(test_questions) * 0.7:
    print("✅ Good! Model mostly follows GRPO format")
else:
    print("⚠️ Warning: Model needs more training on GRPO format")

🧪 Testing GRPO model with structured format validation...


TEST QUESTION #1

❓ Question: Mức phạt cho việc vượt đèn đỏ đối với xe máy là bao nhiêu?

🤖 Model Response:
<start_working_out>
Câu hỏi của người dùng là về mức phạt đối với việc vượt đèn đỏ của xe máy theo quy định của pháp luật Việt Nam. Để trả lời câu hỏi này, tôi sẽ tìm kiếm thông tin trong văn bản pháp luật đã cung cấp, đặc biệt là các điều khoản liên quan đến việc vi phạm giao thông, như hành vi vượt đèn đỏ. 

Trong văn bản pháp luật, có một điểm c khoản 3 Điều 3 quy định rằng "xe mô-tô, xe gắn máy" không được phép vượt đèn đỏ. Tuy nhiên, văn bản không nêu rõ mức phạt cụ thể cho hành vi này. Thông thường, mức phạt đối với hành vi vượt đèn đỏ là từ 1.000.000 đến 2.000.000 đồng, tùy thuộc vào mức độ vi phạm và khả năng của người lái xe. 

Vì vậy, cần tham khảo thêm các quy định khác hoặc các văn bản pháp luật khác để có thông tin cụ thể hơn về mức phạt này.
<end_working_out>

<SOLUTION>Mức phạt cho việc vượt đèn đỏ đối với

## 💾 Save Model

In [21]:
# Optional: Save merged model (full size ~6GB)
model.save_pretrained_merged("grpo_synthetic_merged", tokenizer, save_method="merged_16bit")
print("✅ GRPO synthetic merged model saved to: grpo_synthetic_merged/")
print("🎯 This model now has: Base → GRPO training → Synthetic data SFT")

Configuration saved in grpo_synthetic_merged/config.json


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Checking cache directory for required files...


Unsloth: Copying 2 files from cache to `grpo_synthetic_merged`: 100%|██████████| 2/2 [00:08<00:00,  4.06s/it]


Successfully copied all 2 files from cache to `grpo_synthetic_merged`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:41<00:00, 20.81s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/grpo_synthetic_merged`
✅ GRPO synthetic merged model saved to: grpo_synthetic_merged/
🎯 This model now has: Base → GRPO training → Synthetic data SFT


## 📤 Model Upload & Export

In [25]:
# Upload merged model to HuggingFace Hub (giải pháp cho file lớn!)
# Bước 1: Tạo HuggingFace account tại https://huggingface.co/join
# Bước 2: Tạo token tại https://huggingface.co/settings/tokens (chọn "Write" permission)
# Bước 3: Thêm token vào Kaggle Secrets với key "HF_TOKEN"

import os

if os.path.exists("/kaggle/working"):
    print("🚀 Uploading model to HuggingFace Hub...")
    print("="*70)
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        hf_token = user_secrets.get_secret("HF_TOKEN")
        
        from huggingface_hub import HfApi, login
        
        # Login to HuggingFace
        login(token=hf_token)
        print("✅ Logged in to HuggingFace")
        
        # Thay YOUR_USERNAME bằng username HuggingFace của bạn
        YOUR_HF_USERNAME = "mikeethanh"  # ⚠️ SỬA DÒNG NÀY!
        repo_name = f"{YOUR_HF_USERNAME}/vietnamese-legal-llama3.2-3b-merged-sft-v2"
        
        print(f"\n📤 Uploading to: {repo_name}")
        print("⏳ Đang upload ~6GB, có thể mất 10-15 phút...\n")
        
        # Upload merged model
        if os.path.exists("grpo_synthetic_merged"):
            from huggingface_hub import create_repo, upload_folder
            
            # Create repo (public)
            try:
                create_repo(repo_name, repo_type="model", exist_ok=True)
                print(f"✅ Repository created: https://huggingface.co/{repo_name}")
            except:
                print(f"ℹ️ Repository already exists: https://huggingface.co/{repo_name}")
            
            # Upload folder
            upload_folder(
                folder_path="grpo_synthetic_merged",
                repo_id=repo_name,
                commit_message="Vietnamese Legal AI - Llama 3.2 3B Merged Model",
            )
            
            print("\n" + "="*70)
            print("✅ UPLOAD THÀNH CÔNG!")
            print("="*70)
            print(f"\n📥 Download model về máy bằng cách:")
            print(f"   git clone https://huggingface.co/{repo_name}")
            print(f"\n🌐 Hoặc xem trên web:")
            print(f"   https://huggingface.co/{repo_name}")
            print("\n💡 Model đã public, ai cũng có thể download!")
        else:
            print("⚠️ Folder 'vietnamese_legal_merged' not found!")
            
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n📝 Hướng dẫn fix:")
        print("  1. Tạo account tại: https://huggingface.co/join")
        print("  2. Tạo token tại: https://huggingface.co/settings/tokens")
        print("  3. Kaggle: Add-ons → Secrets → Add 'HF_TOKEN'")
        print("  4. Sửa YOUR_USERNAME trong code")
        
else:
    print("ℹ️ This cell only works on Kaggle")
    print("💡 For local, use: model.push_to_hub() directly")

🚀 Uploading model to HuggingFace Hub...
✅ Logged in to HuggingFace

📤 Uploading to: mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2
⏳ Đang upload ~6GB, có thể mất 10-15 phút...

✅ Repository created: https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✅ UPLOAD THÀNH CÔNG!

📥 Download model về máy bằng cách:
   git clone https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2

🌐 Hoặc xem trên web:
   https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2

💡 Model đã public, ai cũng có thể download!


## 📊 Quantization Export

In [ ]:
# Export to GGUF for deployment
quantization_methods = [
    "q8_0",    # Fast inference, good quality
    "q4_k_m",  # Smaller size, good balance
]

for method in quantization_methods:
    print(f"\n📦 Exporting GRPO synthetic model to {method.upper()}...")
    model.save_pretrained_gguf(
        "grpo_synthetic_model",
        tokenizer,
        quantization_method=method,
    )
    print(f"✅ Exported: grpo_synthetic_model-{method.upper()}.gguf")

print("\n✅ All GGUF exports completed!")
print("🚀 Ready for deployment with Ollama or llama.cpp")

## 🎉 Training Summary & Cleanup

In [ ]:
# Finish WandB run
wandb.finish()

# Clear GPU memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("✅ GRPO Synthetic Data Training completed successfully!")
print("\n📊 FINAL SUMMARY:")
print("="*60)
print(f"🤖 Base Model: mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo")
print(f"📊 Training samples: {len(train_data):,} (synthetic GRPO format)")
print(f"📊 Validation samples: {len(val_data):,}")
print(f"📊 Test samples: {len(test_data):,}")
print(f"⏱️ Training time: ~{trainer_stats.metrics['train_runtime']/60:.1f} minutes")
print(f"🎯 Final eval loss: {eval_results['eval_loss']:.4f}")

print("\n📂 SAVED OUTPUTS:")
print("  ✅ LoRA adapters: grpo_synthetic_lora/")
print("  ✅ Merged model: grpo_synthetic_merged/")
print("  ✅ GGUF models: grpo_synthetic_model-*.gguf")

print("\n🎯 MODEL EVOLUTION COMPLETE:")
print("  1️⃣ Base: Llama-3.2-3B-Instruct")
print("  2️⃣ GRPO: Reinforcement learning với reward functions")
print("  3️⃣ SFT: Synthetic data với structured reasoning format")

print("\n🚀 NEXT STEPS:")
print("  1. Test model on real user queries")
print("  2. Validate GRPO format consistency")
print("  3. Deploy and collect feedback")
print("  4. Iterate with more synthetic data if needed")

print("\n🎉 Training pipeline complete! Model ready for deployment.")